# Semantic Kernel ツール使用例

このドキュメントでは、Semantic Kernel をベースにしたツールを作成するためのコードの概要と説明を提供します。このツールは、Azure AI Search を統合して情報検索を強化した生成（RAG）を実現します。この例では、Azure AI Search インデックスから旅行関連のドキュメントを取得し、ユーザーのクエリをセマンティック検索結果で補強し、詳細な旅行のおすすめ情報をストリーミングする AI エージェントの構築方法を示します。


### パッケージのインポート
以下のコードは必要なパッケージをインポートします:


In [ ]:
import json
import os

from typing import Annotated

from IPython.display import display, HTML

from dotenv import load_dotenv

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent,FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

### セマンティックカーネルとAIサービスの作成

セマンティックカーネルのインスタンスは、非同期のOpenAIチャット補完サービスを使用して作成および設定されます。このサービスは、応答を生成するためにカーネルに追加されます。


In [ ]:
load_dotenv()
# Initialize the asynchronous OpenAI client
client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com/"
)

# Create the OpenAI Chat Completion Service
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

### プロンプトプラグインの定義

PromptPluginは、リトリーバルコンテキストを使用して拡張されたプロンプトを構築する関数を定義するネイティブプラグインです。


In [ ]:
class SearchPlugin:

    def __init__(self, search_client: SearchClient):
        self.search_client = search_client

    @kernel_function(
        name="build_augmented_prompt",
        description="Build an augmented prompt using retrieval context or function results.",
    )
    def build_augmented_prompt(self, query: str, retrieval_context: str) -> str:
        return (
            f"Retrieved Context:\n{retrieval_context}\n\n"
            f"User Query: {query}\n\n"
            "First review the retrieved context, if this does not answer the query, try calling an available plugin functions that might give you an answer. If no context is available, say so."
        )
    
    @kernel_function(
        name="retrieve_documents",
        description="Retrieve documents from the Azure Search service.",
    )
    def get_retrieval_context(self, query: str) -> str:
        results = self.search_client.search(query)
        context_strings = []
        for result in results:
            context_strings.append(f"Document: {result['content']}")
        return "\n\n".join(context_strings) if context_strings else "No results found"

In [ ]:
class WeatherInfoPlugin:
    """A Plugin that provides the average temperature for a travel destination."""

    def __init__(self):
        # Dictionary of destinations and their average temperatures
        self.destination_temperatures = {
            "maldives": "82°F (28°C)",
            "swiss alps": "45°F (7°C)",
            "african safaris": "75°F (24°C)"
        }

    @kernel_function(description="Get the average temperature for a specific travel destination.")
    def get_destination_temperature(self, destination: str) -> Annotated[str, "Returns the average temperature for the destination."]:
        """Get the average temperature for a travel destination."""
        # Normalize the input destination (lowercase)
        normalized_destination = destination.lower()

        # Look up the temperature for the destination
        if normalized_destination in self.destination_temperatures:
            return f"The average temperature in {destination} is {self.destination_temperatures[normalized_destination]}."
        else:
            return f"Sorry, I don't have temperature information for {destination}. Available destinations are: Maldives, Swiss Alps, and African safaris."

## ベクターデータベースの初期化

永続的なストレージを使用して Azure AI Search を初期化し、拡張されたサンプルドキュメントを追加します。Azure AI Search は、正確な回答を生成するためのコンテキストを提供するドキュメントの保存と取得に使用されます。


In [ ]:
# Initialize Azure AI Search with persistent storage
search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = "travel-documents"

search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Define the index schema
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="content", type=SearchFieldDataType.String)
]

index = SearchIndex(name=index_name, fields=fields)

# Check if index already exists if not, create it
try:
    existing_index = index_client.get_index(index_name)
    print(f"Index '{index_name}' already exists, using the existing index.")
except Exception:
    # Create the index if it doesn't exist
    print(f"Creating new index '{index_name}'...")
    index_client.create_index(index)


# Enhanced sample documents
documents = [
    {"id": "1", "content": "Contoso Travel offers luxury vacation packages to exotic destinations worldwide."},
    {"id": "2", "content": "Our premium travel services include personalized itinerary planning and 24/7 concierge support."},
    {"id": "3", "content": "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage."},
    {"id": "4", "content": "Popular destinations include the Maldives, Swiss Alps, and African safaris."},
    {"id": "5", "content": "Contoso Travel provides exclusive access to boutique hotels and private guided tours."}
]

# Add documents to the index
search_client.upload_documents(documents)

In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[SearchPlugin(search_client=search_client), WeatherInfoPlugin()],
    name="TravelAgent",
    instructions="Answer travel queries using the provided tools and context. If context is provided, do not say 'I have no context for that.'",
)

### ストリーミング呼び出しでエージェントを実行する

メインの非同期ループは会話用のスレッドを作成し、各ユーザー入力ごとにエージェントが取得コンテキストを確認できるようにします。ユーザーメッセージも追加され、その後ストリーミングを使用してエージェントが呼び出されます。出力はストリーミングされると同時に表示されます。


In [ ]:
async def main():
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Can you explain Contoso's travel insurance coverage?",
        "What is the average temperature of the Maldives?",
        "What is a good cold destination offered by Contoso and what is it average temperature?",
    ]

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


了解しました。翻訳を始めます。

# はじめに

このドキュメントでは、@@INLINE_CODE_0@@を使用してアプリケーションを構築する方法について説明します。以下の手順に従うことで、プロジェクトを迅速に開始できます。

## 必要条件

- @@INLINE_CODE_1@@がインストールされていることを確認してください。
- 基本的な@@INLINE_CODE_2@@の知識があること。

## インストール

1. @@INLINE_CODE_3@@を使用して依存関係をインストールします。
   ```bash
   ```text
User: 
Can you explain Contoso's travel insurance coverage?

Function Calls (click to expand)

Calling function: retrieve_documents({"query": "Contoso travel insurance coverage"})

Function Result:

Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.

Document: Contoso Travel provides exclusive access to boutique hotels and private guided tours.

Document: Our premium travel services include personalized itinerary planning and 24/7 concierge support.

TravelAgent:

Contoso's travel insurance coverage includes the following:

1. **Medical Emergencies**: Coverage for unforeseen medical issues that may arise while traveling.
2. **Trip Cancellations**: Protection in case you need to cancel your trip for covered reasons.
3. **Lost Baggage**: Compensation for baggage that is lost during your trip.

If you need more specific details about the policy, it would be best to contact Contoso directly or refer to their official documentation.
```
   ```
2. プロジェクトを初期化します。
   ```bash
   @@CODE_BLOCK_1@@
   ```

[!NOTE] この手順は、最新バージョンの@@INLINE_CODE_4@@を使用していることを前提としています。

## 使用方法

以下のコードスニペットは、基本的な使用例を示しています。

```python
@@CODE_BLOCK_2@@
```

[!TIP] 必要に応じて@@INLINE_CODE_5@@をカスタマイズしてください。

## よくある問題

### 問題1: インストール中にエラーが発生する

[!WARNING] このエラーは、@@INLINE_CODE_6@@が正しく設定されていない場合に発生する可能性があります。

解決方法:
1. @@INLINE_CODE_7@@を確認してください。
2. 必要に応じて再インストールを試みてください。

### 問題2: 実行時にパフォーマンスが低下する

[!CAUTION] 大量のデータを処理する場合、@@INLINE_CODE_8@@の設定を調整する必要があります。

解決方法:
- @@INLINE_CODE_9@@を最適化する。
- 必要に応じてキャッシュを利用する。

## 結論

このドキュメントでは、@@INLINE_CODE_0@@を使用してアプリケーションを構築する基本的な手順を説明しました。さらなる情報が必要な場合は、公式ドキュメントを参照してください。

[!IMPORTANT] 常に最新のセキュリティアップデートを適用することを忘れないでください。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤った解釈について、当方は責任を負いません。
